In [30]:
import stdpopsim
import tskit
import msprime
import numpy as np
import io
import os
import itertools
import subprocess
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
import warnings
warnings.filterwarnings('ignore')

# 设置随机种子
np.random.seed(42)

# 配置matplotlib
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print(f"stdpopsim version: {stdpopsim.__version__}")
print(f"tskit version: {tskit.__version__}")
print(f"msprime version: {msprime.__version__}")

stdpopsim version: 0.3.0
tskit version: 0.6.4
msprime version: 1.3.4


In [33]:
# 1. 物种与染色体片段（stdpopsim 仅提供参数）
species = stdpopsim.get_species("HomSap")
contig = species.get_contig("chr22") 

# 2. 自定义分化模型（msprime.Demography）
generation_time = 25
T_split = 1_000_000 / generation_time  # 世代数

demography = msprime.Demography()
demography.add_population(name="PopA", initial_size=10_000)
demography.add_population(name="PopB", initial_size=10_000)
demography.add_population(name="Ancestral", initial_size=10_000)
demography.add_population_split(time=T_split, derived=["PopA", "PopB"], ancestral="Ancestral")

# 3. 采样列表（msprime.SampleSet）
samples = [
    msprime.SampleSet(1000, population="PopA", time=0),
    msprime.SampleSet(1000, population="PopB", time=0),
    msprime.SampleSet(10,   population="Ancestral", time=T_split),
]

# 4. 模拟祖先谱系
ts_anc = msprime.sim_ancestry(
    samples=samples,
    demography=demography,
    sequence_length=contig.length,
    recombination_rate=contig.recombination_map,
    random_seed=42,
)

# 5. 叠加突变
ts = msprime.sim_mutations(
    ts_anc,
    rate=contig.mutation_rate,
    model="jc69", # Choose from ['binary', 'blosum62', 'infinite_alleles', 'jc69', 'pam']
    random_seed=42,
)

# 6. 结果查看
print(f"模拟完成：{ts.num_mutations} 个突变，{ts.num_samples} 条线")
print(f"树序列文件大小：{ts.num_trees} 棵树，{ts.sequence_length/1e6:.1f} Mb")

模拟完成：515326 个突变，4020 条线
树序列文件大小：760997 棵树，50.8 Mb


In [36]:
print(f"总样本数: {ts.num_samples:,}")
print(f"树数量: {ts.num_trees:,}")
print(f"突变数: {ts.num_mutations:,}")
print(f"序列长度: {ts.sequence_length:,.0f} bp")

# 基本统计
diversity_s2 = ts.diversity()
tajimas_d_s2 = ts.Tajimas_D()
print(f"\n核苷酸多样性 (π): {diversity_s2:.6f}")
print(f"Tajima's D: {tajimas_d_s2:.6f}")

pop_labels = ["PopA", "PopB", "Ancestor"]
sample_sets = [ts.samples(population=i, time=None) for i in range(3)]
indexes = list(itertools.combinations(range(3), 2))
fst_pairs = ts.Fst(sample_sets, indexes=indexes)

fst_matrix = np.zeros((3, 3))
for idx, (i, j) in enumerate(indexes):
    fst_matrix[i, j] = fst_pairs[idx]
    fst_matrix[j, i] = fst_pairs[idx]

fst_df = pd.DataFrame(fst_matrix, index=pop_labels, columns=pop_labels)
print("\nFST 3×3 矩阵：")
print(fst_df)

总样本数: 4,020
树数量: 760,997
突变数: 515,326
序列长度: 50,818,468 bp

核苷酸多样性 (π): 0.001024
Tajima's D: -0.284639

FST 3×3 矩阵：
              PopA      PopB  Ancestor
PopA      0.000000  0.497741  0.332424
PopB      0.497741  0.000000  0.333563
Ancestor  0.332424  0.333563  0.000000


In [ ]:
vcf_names = [f"{pop_labels[ts.node(ts.individual(i).nodes[0]).population]}_{i}"
             for i in range(ts.num_individuals)]

popA_inds = [i for i in range(ts.num_individuals)
             if ts.node(ts.individual(i).nodes[0]).population == 0]
popB_inds = [i for i in range(ts.num_individuals)
             if ts.node(ts.individual(i).nodes[0]).population == 1]
anc_inds  = [i for i in range(ts.num_individuals)
             if ts.node(ts.individual(i).nodes[0]).population == 2]

print("\n个体计数：")
print("PopA:", len(popA_inds), "PopB:", len(popB_inds), "Ancestor:", len(anc_inds))


个体计数：
PopA: 1000 PopB: 1000 Ancestor: 10


In [39]:
def write_vcf_subset(ts, ind_list, out_vcf_gz, contig_name="chr1"):
    with subprocess.Popen(["bgzip", "-c"],
                          stdin=subprocess.PIPE,
                          stdout=open(out_vcf_gz, "wb")) as bgz:
        txt = io.StringIO()
        ts.write_vcf(txt, individuals=ind_list, contig_id=contig_name)
        bgz.communicate(input=txt.getvalue().encode())

out_dir = "/home/qmtang/mnt_qmtang/EvoFill/data/sim_0910/"
write_vcf_subset(ts, popA_inds,  os.path.join(out_dir, "PopA.vcf.gz"))
write_vcf_subset(ts, popB_inds,  os.path.join(out_dir, "PopB.vcf.gz"))
write_vcf_subset(ts, anc_inds,   os.path.join(out_dir, "Ancestor.vcf.gz"))

In [40]:
for pop in ["PopA", "PopB", "Ancestor"]:
    vcf = os.path.join(out_dir,f"{pop}.vcf.gz")
    lines = int(subprocess.check_output(f"bcftools view -H {vcf} | wc -l", shell=True))
    print(f"{vcf}  变异行数: {lines}")

/home/qmtang/mnt_qmtang/EvoFill/data/sim_0910/PopA.vcf.gz  变异行数: 512740
/home/qmtang/mnt_qmtang/EvoFill/data/sim_0910/PopB.vcf.gz  变异行数: 512740
/home/qmtang/mnt_qmtang/EvoFill/data/sim_0910/Ancestor.vcf.gz  变异行数: 512740
